In [84]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, LSTM, Conv2D, MaxPooling1D, MaxPooling2D
from sklearn.preprocessing import StandardScaler
from keras.optimizers import Adam, SGD
import pandas as pd
import numpy as np

In [85]:
dataset_path = '../datasets/synergy-final-iter1-processed/'

device = 'xdk_1'

df = pd.read_pickle(dataset_path + device + '.p')
df_labels = pd.read_pickle(dataset_path + device + '_labels.p')

# filter activities
use_activities = [0,1,2,3,4,5,6,7]
num_classes = len(use_activities)
df_labels = df_labels.loc[df_labels.label.isin(use_activities)]

In [86]:
columns = ['accel_x', 'accel_y', 'accel_z']

scaler = StandardScaler()

df[columns] = scaler.fit_transform(df[columns])

In [87]:
from keras.preprocessing.sequence import pad_sequences

def extract_window_data(window_id):
    segment = df.loc[df.id == window_id]
    data = segment[columns].values.T
    return pad_sequences(data,
                         maxlen=50,
                         dtype='float64',
                         padding='post',
                         value=0)

x = np.array([extract_window_data(id) for id in df_labels.index])
y = df_labels[['label']].values

In [88]:
x.shape

(578, 3, 50)

In [89]:
train_test_split = np.random.rand(len(x)) < 0.70
x_train = x[train_test_split]
y_train = y[train_test_split]
x_test = x[~train_test_split]
y_test = y[~train_test_split]

y_train = keras.utils.to_categorical(y_train, num_classes=num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes=num_classes)

In [94]:
batch_size = 32
timesteps = 3
data_dim = 50

In [95]:
import math

# x_train = x_train[:math.floor(len(x_train) / batch_size) * batch_size]
# x_test = x_test[:math.floor(len(x_test) / batch_size) * batch_size]
# y_train = y_train[:math.floor(len(y_train) / batch_size) * batch_size]
# y_test = y_test[:math.floor(len(y_test) / batch_size) * batch_size]

In [103]:
model = Sequential()
model.add(LSTM(128, return_sequences=True, stateful=False,
               input_shape=(timesteps, data_dim)))
model.add(MaxPooling1D(3))
model.add(Dropout(1))
model.add(LSTM(128, return_sequences=True, stateful=False))
model.add(Dropout(1))
model.add(LSTM(128, stateful=False))
model.add(Dropout(1))
model.add(Dense(num_classes, activation='softmax'))

opt = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=batch_size, epochs=50)

score = model.evaluate(x_test, y_test, batch_size=batch_size)
score

Train on 416 samples, validate on 162 samples
Epoch 1/50
416/416 [==============================] - 1s - loss: 2.0792 - acc: 0.1298 - val_loss: 2.0802 - val_acc: 0.1111
Epoch 2/50
416/416 [==============================] - 0s - loss: 2.0716 - acc: 0.2043 - val_loss: 2.0813 - val_acc: 0.1235
Epoch 3/50
416/416 [==============================] - 0s - loss: 2.0587 - acc: 0.2163 - val_loss: 2.0828 - val_acc: 0.1358
Epoch 4/50
416/416 [==============================] - 0s - loss: 2.0254 - acc: 0.2284 - val_loss: 2.0877 - val_acc: 0.1358
Epoch 5/50
416/416 [==============================] - 0s - loss: 1.9296 - acc: 0.2572 - val_loss: 2.1184 - val_acc: 0.1543
Epoch 6/50
416/416 [==============================] - 0s - loss: 1.7653 - acc: 0.3654 - val_loss: 2.2387 - val_acc: 0.1605
Epoch 7/50
416/416 [==============================] - 0s - loss: 1.5225 - acc: 0.4712 - val_loss: 2.3121 - val_acc: 0.1790
Epoch 8/50
416/416 [==============================] - 0s - loss: 1.2638 - acc: 0.5264 - val_l

[5.529796138221835, 0.19135802469135801]

In [99]:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 10

model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
# model.add(LSTM(128, stateful=False,
#                input_shape=(timesteps, data_dim)))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 input_shape=(timesteps, data_dim),
                 strides=1))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          epochs=epochs,
          batch_size=batch_size)
score = model.evaluate(x_test, y_test, batch_size=batch_size)
score

ValueError: Negative dimension size caused by subtracting 3 from 1 for 'max_pooling1d_3/MaxPool' (op: 'MaxPool') with input shapes: [?,1,1,250].

In [58]:
x_train

array([[[ 0.09348373,  0.09348373,  0.46311822, ..., -1.01541975,
         -0.27615077, -0.27615077],
        [-1.21433523, -1.21433523, -0.05500802, ..., -1.98722003,
          1.10431918,  1.10431918],
        [ 0.60179186,  0.60179186,  0.60179186, ...,  0.60179186,
         -0.73721969, -0.73721969]],

       [[ 0.46311822,  0.46311822,  0.09348373, ..., -1.38505424,
         -1.38505424,  0.        ],
        [ 0.33143438,  0.33143438, -0.44145042, ...,  0.33143438,
          0.33143438,  0.        ],
        [ 2.2755563 ,  2.2755563 ,  0.60179186, ...,  0.26703897,
          0.26703897,  0.        ]],

       [[-1.75468873, -1.75468873,  0.46311822, ..., -1.38505424,
          0.46311822,  0.46311822],
        [-0.44145042, -0.44145042,  0.33143438, ..., -0.44145042,
         -0.05500802, -0.05500802],
        [-0.73721969, -0.73721969,  0.26703897, ...,  1.27129763,
          0.60179186,  0.60179186]],

       ..., 
       [[ 0.09348373,  0.09348373,  0.46311822, ..., -1.7546887